In [1]:
import os
import subprocess
import sys
from pyspark.sql import SparkSession

# Get the Spark home directory
spark_home = os.environ.get('SPARK_HOME', '/opt/spark')

# Define all required JARs
jars = [
    f"{spark_home}/jars/delta-core_2.12-2.4.0.jar",
    f"{spark_home}/jars/delta-storage-2.4.0.jar",
    f"{spark_home}/jars/hadoop-aws-3.3.2.jar",
    f"{spark_home}/jars/aws-java-sdk-bundle-1.12.261.jar"
]

# Create SparkSession using the builder pattern
builder = (SparkSession.builder
           .appName("DeltaExample")
           .master("local[*]")
           # Add debug configurations
           .config("spark.hadoop.fs.s3a.connection.maximum", "1")
           .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
           .config("spark.hadoop.fs.s3a.connection.timeout", "5000")
           .config("spark.hadoop.fs.s3a.impl.disable.cache", "true")
           .config("spark.hadoop.fs.s3a.debug.detailed.exceptions", "true")
           # Add jars directly
           .config("spark.jars", ",".join(jars))
           .config("spark.driver.extraClassPath", ",".join(jars))
           .config("spark.executor.extraClassPath", ",".join(jars))
           # Delta Lake configurations
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
           # S3/MinIO configurations
           .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
           .config("spark.hadoop.fs.s3a.path.style.access", "true")
           .config("fs.s3a.metrics.enabled", "false")
           .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
           .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
           .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
           # Additional Delta Lake configurations
           .config("spark.delta.logStore.class", "io.delta.storage.S3SingleDriverLogStore")
           .config("spark.hadoop.fs.s3a.fast.upload", "true")
           .config("spark.hadoop.fs.s3a.multipart.size", "104857600")
           .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse"))

# Stop any existing session
if 'spark' in locals():
    spark.stop()


# Create the session
spark = builder.enableHiveSupport().getOrCreate()

# Initialize Delta Lake settings
spark.sql("SET spark.databricks.delta.formatCheck.enabled=false")

# Access the SparkContext
# sc = spark.sparkContext

# Set the log level to INFO
# sc.setLogLevel("DEBUG")

# Test DataFrame
data = [(1, "John"), (2, "Jane")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

# First verify the S3 connection by listing the bucket
try:
    # Try to write to a simple parquet file first to test S3 connection
    print("Testing S3 connection with parquet write...")
    df.write.format("parquet").mode("overwrite").save("s3a://wba/test.parquet")
    print("S3 connection successful")

    print("Attempting to write Delta table...")
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .option("delta.compatibility.symlinkFormatManifest.enabled", "false") \
        .save("s3a://wba/example-table")
    print("Successfully wrote Delta table")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nTrying local filesystem instead...")
    try:
        local_path = "/tmp/test-delta-table"
        df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(local_path)
        print(f"Successfully wrote to {local_path}")
    except Exception as local_e:
        print(f"Error writing to local filesystem: {str(local_e)}")

25/02/20 19:02:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Jane|
+---+----+

Testing S3 connection with parquet write...


25/02/20 19:02:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


S3 connection successful
Attempting to write Delta table...


25/02/20 19:02:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Successfully wrote Delta table
